In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint as pp
import csv
import datetime
from dateutil.parser import parse
from pandas.tools.plotting import scatter_matrix
from patsy import dmatrices
import statsmodels.formula.api as smf
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm
import seaborn as sns
import statsmodels.api as sm
import matplotlib as mpl
from numpy.random import randn
from scipy import stats
from pprint import pprint as pp
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.learning_curve import learning_curve
from sklearn.cross_validation import cross_val_score
import urllib2
from pandas import Series
from pandas import merge, concat
%matplotlib inline

sns.set_palette("deep", desat=.8)
sns.set_context(rc={"figure.figsize": (14, 8)})

In [103]:
url           = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/cleveland.data"

newrl         = urllib2.urlopen(url)
newdata_df    = pd.read_csv(newrl, header=-1)

newdata_df.head()

,0
0,1 0 63 1 -9 -9 -9
1,-9 1 145 1 233 -9 50 20
2,1 -9 1 2 2 3 81 0
3,0 0 0 0 1 10.5 6 13
4,150 60 190 90 145 85 0 0


In [104]:
cleve_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/cleveland.data"
newrl = urllib2.urlopen(cleve_url)
colus = list(range(1,15))
cleve_df = pd.read_csv("switz.data", header=None, delimiter="name\n")
# id_data = cleve_df[0].str.strip("[]").str.split(",").tolist()
# df_id = pd.DataFrame(id_data, columns=colus, index=cleve_df.index, dtype=int)
# cleve_df = cleve_df.join(df_id)
cleve_df.columns = ["name"]
r1 = cleve_df[0::10]
r2 = cleve_df[1::10]
r3 = cleve_df[2::10]
r4 = cleve_df[3::10]
r5 = cleve_df[4::10]
r6 = cleve_df[5::10]
r7 = cleve_df[6::10]
r8 = cleve_df[7::10]
r9 = cleve_df[8::10]
r10 = cleve_df[9::10]
# cleve_df

In [105]:
cleve_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/cleveland.data"
newrl = urllib2.urlopen(cleve_url)
cleve_df = pd.read_csv(newrl, header=None)
# cleve_df = read_table(newrl, index_col=[0,1,2])
cleve_df.head()

,0
0,1 0 63 1 -9 -9 -9
1,-9 1 145 1 233 -9 50 20
2,1 -9 1 2 2 3 81 0
3,0 0 0 0 1 10.5 6 13
4,150 60 190 90 145 85 0 0


In [540]:
def all_models(df):
    models = [LogisticRegression(), GaussianNB(), DecisionTreeClassifier(max_depth=6), RandomForestClassifier(), KNeighborsClassifier(n_neighbors=11), SVC()]
    data = []
    x_all = df.iloc[:,1:]
    y_all = df.iloc[:,0]
    x_scaled = preprocessing.scale(x_all)
    for model in models:
        m = model
        avg_acc = np.mean(cross_val_score(m, x_scaled, y_all, scoring='recall'))
        avg_prec = np.mean(cross_val_score(m, x_scaled, y_all, scoring='precision'))
        avg_rec = np.mean(cross_val_score(m, x_scaled, y_all, scoring='recall'))
        avg_f1 = np.mean(cross_val_score(m, x_scaled, y_all, scoring='f1'))
        
        data.append([avg_acc,avg_prec,avg_rec,avg_f1])
    index=['Logistic', 'Gaussian', 'DecisionTree', 'RandomForest', 'KNN', 'SVC']
    columns = ['accuracy', 'precision', 'recall', 'F1']
    df = pd.DataFrame(data, index=index, columns=columns)
    return df



In [543]:
tdf = pd.read_csv("ch.csv")
tdf = tdf.replace(-9, np.nan)
tdf = tdf.fillna(tdf.mean())
tdf["num"] = tdf["num"].replace(2,1).replace(3,1).replace(4,1)
drops = ["painloc", "painexer","years", "relrest", "smoke", "rldv5", "id", "ccf", "lvx1", "pncaden", "exeref", "earlobe", "thalpul", "exerwm", "restwm", "exerckm", "restckm", "restef", "lvx2", "lvx3", "lvx4", "lvf", "cathef", "junk","ekgmo", "ekgday", "proto", "dig", "prop", "nitr", "pro", "diuretic", "thalsev", "cmo", "cday", "lmt", "ladprox", "laddist", "diag","cxmain", "ramus", "om1", "om2", "rcaprox", "rcadist"]
tdf = tdf.drop(drops, axis=1)
cols = tdf.columns.tolist()
cols = cols[-1:] + cols[:-1]
tdf = tdf[cols]

# dummies = pd.get_dummies(tdf['location'])

# atom_col = [c for c in dummies.columns if '*' not in c]

# for col in atom_col:
#     tdf[col] = dummies[[c for c in dummies.columns if col in c]].sum(axis=1) 


tdf = tdf.drop(["location"], axis=1)
tdf.describe()
tdf.head(10)

# tdf.mode()
# tdf.shape
# tdf.mean()
# tdf.dtypes

thing = all_models(tdf)
thing

,accuracy,precision,recall,F1
Logistic,0.871120,0.934029,0.871120,0.896510
Gaussian,0.621682,0.916284,0.621682,0.677467
DecisionTree,0.827036,0.929861,0.879667,0.877008
RandomForest,0.914530,0.938084,0.940171,0.934720
KNN,1.000000,0.935134,1.000000,0.966449
SVC,1.000000,0.935134,1.000000,0.966449


In [ ]:
tdf = pd.read_csv("ch.csv")
tdf = tdf.replace(-9, np.nan)
tdf = tdf.fillna(tdf.mean())
tdf["num"] = tdf["num"].replace(2,1).replace(3,1).replace(4,1)
drops = ["painloc", "painexer","years", "relrest", "smoke", "rldv5", "id", "ccf", "lvx1", "pncaden", "exeref", "earlobe", "thalpul", "exerwm", "restwm", "exerckm", "restckm", "restef", "lvx2", "lvx3", "lvx4", "lvf", "cathef", "junk","ekgmo", "ekgday", "proto", "dig", "prop", "nitr", "pro", "diuretic", "thalsev", "cmo", "cday", "lmt", "ladprox", "laddist", "diag","cxmain", "ramus", "om1", "om2", "rcaprox", "rcadist"]
tdf = tdf.drop(drops, axis=1)
cols = tdf.columns.tolist()
cols = cols[-1:] + cols[:-1]
tdf = tdf[cols]

dummies = pd.get_dummies(tdf['location'])

atom_col = [c for c in dummies.columns if '*' not in c]

for col in atom_col:
    tdf[col] = dummies[[c for c in dummies.columns if col in c]].sum(axis=1) 


tdf = tdf.drop(["location"], axis=1)
tdf.describe()
tdf.head(10)

# tdf.mode()
# tdf.shape
# tdf.mean()
# tdf.dtypes

thing = all_models(tdf)
thing

In [525]:
x_all = tdf.iloc[:,1:]
y_all = tdf.iloc[:,0]
tdf.head()
y_all.head()
# x_all.head()

array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2],
       [3, 3, 3],
       [4, 4, 4],
       [5, 5, 5],
       [6, 6, 6],
       [7, 7, 7],
       [8, 8, 8],
       [9, 9, 9]])

In [531]:
x_scaled = preprocessing.scale(x_all)
data = np.array([np.arange(10)]*3).T
data

array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2],
       [3, 3, 3],
       [4, 4, 4],
       [5, 5, 5],
       [6, 6, 6],
       [7, 7, 7],
       [8, 8, 8],
       [9, 9, 9]])

In [518]:
train, test = train_test_split(tdf, train_size = 0.8)
x_train  = train.iloc[:,1:]
y_train  = train.iloc[:,0]
x_test   = test.iloc[:,1:]
y_test   = test.iloc[:,0]

In [519]:
model = LogisticRegression()
avg_acc = np.mean(cross_val_score(model, x_scaled, y_all, scoring='recall'))
avg_precision = np.mean(cross_val_score(model, x_scaled, y_all, scoring='precision'))
avg_recall = np.mean(cross_val_score(model, x_scaled, y_all, scoring='recall'))
avg_f1 = np.mean(cross_val_score(model, x_scaled, y_all, scoring='f1'))
print "Accuracy: %f" % avg_acc
print "Precision: %f" % avg_precision
print "Recall: %f" % avg_recall
print "F1: %f" % avg_f1

Accuracy: 0.871120
Precision: 0.934029
Recall: 0.871120
F1: 0.896510


In [520]:
model = GaussianNB()
avg_acc = np.mean(cross_val_score(model, x_scaled, y_all, scoring='recall'))
avg_precision = np.mean(cross_val_score(model, x_scaled, y_all, scoring='precision'))
avg_recall = np.mean(cross_val_score(model, x_scaled, y_all, scoring='recall'))
avg_f1 = np.mean(cross_val_score(model, x_scaled, y_all, scoring='f1'))
print "Accuracy: %f" % avg_acc
print "Precision: %f" % avg_precision
print "Recall: %f" % avg_recall
print "F1: %f" % avg_f1

Accuracy: 0.621682
Precision: 0.916284
Recall: 0.621682
F1: 0.677467


In [521]:
model = DecisionTreeClassifier(max_depth=6)
avg_acc = np.mean(cross_val_score(model, x_scaled, y_all, scoring='recall'))
avg_precision = np.mean(cross_val_score(model, x_scaled, y_all, scoring='precision'))
avg_recall = np.mean(cross_val_score(model, x_scaled, y_all, scoring='recall'))
avg_f1 = np.mean(cross_val_score(model, x_scaled, y_all, scoring='f1'))
print "Accuracy: %f" % avg_acc
print "Precision: %f" % avg_precision
print "Recall: %f" % avg_recall
print "F1: %f" % avg_f1

Accuracy: 0.844579
Precision: 0.940278
Recall: 0.888439
F1: 0.886398


In [522]:
model = RandomForestClassifier()
avg_acc = np.mean(cross_val_score(model, x_all, y_all, scoring='recall'))
avg_precision = np.mean(cross_val_score(model, x_all, y_all, scoring='precision'))
avg_recall = np.mean(cross_val_score(model, x_all, y_all, scoring='recall'))
avg_f1 = np.mean(cross_val_score(model, x_all, y_all, scoring='f1'))
print "Accuracy: %f" % avg_acc
print "Precision: %f" % avg_precision
print "Recall: %f" % avg_recall
print "F1: %f" % avg_f1

Accuracy: 0.948268
Precision: 0.939335
Recall: 0.956815
F1: 0.939271


In [523]:
model = KNeighborsClassifier(n_neighbors=11)
avg_acc = np.mean(cross_val_score(model, x_scaled, y_all, scoring='recall'))
avg_precision = np.mean(cross_val_score(model, x_scaled, y_all, scoring='precision'))
avg_recall = np.mean(cross_val_score(model, x_scaled, y_all, scoring='recall'))
avg_f1 = np.mean(cross_val_score(model, x_scaled, y_all, scoring='f1'))
print "Accuracy: %f" % avg_acc
print "Precision: %f" % avg_precision
print "Recall: %f" % avg_recall
print "F1: %f" % avg_f1

Accuracy: 1.000000
Precision: 0.935134
Recall: 1.000000
F1: 0.966449


In [524]:
model = SVC()
avg_acc = np.mean(cross_val_score(model, x_scaled, y_all, scoring='recall'))
avg_precision = np.mean(cross_val_score(model, x_scaled, y_all, scoring='precision'))
avg_recall = np.mean(cross_val_score(model, x_scaled, y_all, scoring='recall'))
avg_f1 = np.mean(cross_val_score(model, x_scaled, y_all, scoring='f1'))
print "Accuracy: %f" % avg_acc
print "Precision: %f" % avg_precision
print "Recall: %f" % avg_recall
print "F1: %f" % avg_f1

Accuracy: 1.000000
Precision: 0.935134
Recall: 1.000000
F1: 0.966449
